# Extraction of Latitude Longititude using Googlemaps API

In [2]:
import pandas as pd
import numpy as np
import bs4 as BeautifulSoup
import requests
import urllib2
from ast import literal_eval

#### loaded the data of latitude and longitude data extracted using google maps api from "Capstone-Part2 notebook". As it retrieves the data in json google format. Saved them as it it.

As its stored as list in the columns, but when it exported to dataframe it will automatically covert the values into string rather than list as it is to work with. Geo_All column has data in different formats.

In [3]:
df = pd.read_csv('latlng_30dec2016.csv', na_values='[]')
del df['check']

In [4]:
pd.set_option('display.max_columns', 6000)
pd.set_option('display.max_rows', 6000)

In [5]:
#combining part_1 and Geo columns together
df['GeoAll'] = df[df.columns[3:5]].apply(lambda x: ''.join(x.dropna()),axis=1)

### Challenges with Google maps API:
Latitude and longitude data has been retrieved using google api using its key which is been generated and it only allsows developer to freely download 2500 locations data per 24 hour and that's the free allowance per day for google maps API. I had to wait for another 24 hour to get the data for remaining locations as I have nearly 5000+ locations data.

Neary 419 locations hasn't retrieved any data because either the location details were wrongly entered or its something like 100miles from ocean or 100miles from cc airport which is not recognised by google maps api. Finally after so many intervals and iterations managed to get 4900 rows of data using google maps api.

Next difficulty was when its been stored into .csv format and loaded back into dataframe it automatically converts the list format of json output into simply string which is rather surprise and had to search lot to figure out how to convert those string into just back to original list so that its easy to extract just latitude and longitude and proper country names.

Finally there was library called 'literal_eval' which will convert the string of list into just plain list for me to work with the json again and retrieve lat,lng and country names. countrylatlng() function has been written below to achieve the same.

In [7]:
df['GeoAll'][0]

"[{u'geometry': {u'location_type': u'APPROXIMATE', u'bounds': {u'northeast': {u'lat': 53.74281089999999, u'lng': 0.1460968}, u'southwest': {u'lat': 53.52514069999999, u'lng': -0.699125}}, u'viewport': {u'northeast': {u'lat': 53.74281089999999, u'lng': 0.1460968}, u'southwest': {u'lat': 53.52514069999999, u'lng': -0.699125}}, u'location': {u'lat': 53.63397579999999, u'lng': -0.2765141}}, u'address_components': [{u'long_name': u'Humber', u'types': [u'establishment', u'natural_feature'], u'short_name': u'Humber'}, {u'long_name': u'United Kingdom', u'types': [u'country', u'political'], u'short_name': u'GB'}], u'place_id': u'ChIJX922qfPqeEgRGWU6pzQkNJE', u'formatted_address': u'Humber, United Kingdom', u'types': [u'establishment', u'natural_feature']}]"

In [576]:
#function to retrieve latitude, longitude and country name
def countrylatlng(tt1):
    #print tt1
    if type(tt1)!=float:
        if len(tt1)>2:
            tt1=literal_eval(tt1)
            
            add_c = tt1[0]['address_components']
            #print type(add_c), len(add_c)
            lat = float(tt1[0]['geometry']['location']['lat'])
            lng = float(tt1[0]['geometry']['location']['lng'])

            wanted_keys=['types']
            
            #print "add_c",add_c
            country = "None"

            for i in range(len(add_c)):
                old_dict=add_c[i]
                #print old_dict
                new_dict = {k: old_dict[k] for k in set(wanted_keys) & set(old_dict.keys())}
                #print new_dict
                if str(new_dict.values()).find('country') <> -1:
                    #print i,"----"
                    country = add_c[i]['long_name']
                    #print country
                    #print "-----"
                elif str(new_dict.values()).find('postal_code') == -1:
                    #print i,"*****"
                    #print "xxxx"
                    country = add_c[i]['long_name']
                    #print "11111",add_c[i]['long_name']
                    #print country
                    #print "******"
                #print country

                
            #print i
            #print lat, lng, country    
            return (lat, lng, country)
        else:
            #print tt1
            return (tt1, tt1, tt1)
    
    else:
        #print "finalelse"
        return (tt1, tt1, tt1)
        

In [577]:
countrylatlng(df['part_1'][5])

(-14.5994134, -28.6731465, u'Atlantic Ocean')

In [579]:
df['latlngcountry']=df['GeoAll'].map(countrylatlng)

In [580]:
df['latitude']=df['latlngcountry'].map(lambda x: x[0])
df['longititude']=df['latlngcountry'].map(lambda x: x[1])
df['loccountry']=df['latlngcountry'].map(lambda x: x[2])

In [581]:
df.head()

,Unnamed: 0,Date,Location,part_1,Geo,Geo_1,GeoAll,latlngcountry,latitude,longititude,loccountry
0,0,8/24/1921,"River Humber, England",[{u'geometry': {u'location_type': u'APPROXIMAT...,NaN,NaN,[{u'geometry': {u'location_type': u'APPROXIMAT...,"(53.6339758, -0.2765141, United Kingdom)",53.634,-0.276514,United Kingdom
1,1,9/6/1921,"Paris, France",[{u'geometry': {u'location_type': u'APPROXIMAT...,NaN,NaN,[{u'geometry': {u'location_type': u'APPROXIMAT...,"(48.856614, 2.3522219, France)",48.8566,2.35222,France
2,2,12/5/1921,"Near Murchinson River, Australia",[{u'geometry': {u'location_type': u'APPROXIMAT...,NaN,NaN,[{u'geometry': {u'location_type': u'APPROXIMAT...,"(-25.8742799, 118.8406464, Australia)",-25.8743,118.841,Australia
3,3,1/14/1922,"Paris, France",[{u'geometry': {u'location_type': u'APPROXIMAT...,NaN,NaN,[{u'geometry': {u'location_type': u'APPROXIMAT...,"(48.856614, 2.3522219, France)",48.8566,2.35222,France
4,4,2/21/1922,"Near Norfolk, Virginia",[{u'geometry': {u'location_type': u'APPROXIMAT...,NaN,NaN,[{u'geometry': {u'location_type': u'APPROXIMAT...,"(36.8507689, -76.2858726, United States)",36.8508,-76.2859,United States


In [605]:
df['loccountry'][336]='Mexico'
df['loccountry'][439]='Australia'
df['loccountry'][604]='Norway'

In [606]:
print len(df[df['latitude']==''])
print len(df[df['longititude']==''])
print len(df[df['loccountry']==''])

419
419
419


In [619]:
df_final =df.drop(['Unnamed: 0','part_1','Geo','Geo_1'], axis=1)

In [620]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5367 entries, 0 to 5366
Data columns (total 7 columns):
Date             5367 non-null object
Location         5367 non-null object
GeoAll           5367 non-null object
latlngcountry    5367 non-null object
latitude         5367 non-null object
longititude      5367 non-null object
loccountry       5367 non-null object
dtypes: object(7)
memory usage: 293.6+ KB


In [624]:
# saving the final output into .csv to work with later in model
df_final.to_csv('Final_latlngcountry_31Dec.csv', encoding='utf-8')

In [625]:
df_final.head()

,Date,Location,GeoAll,latlngcountry,latitude,longititude,loccountry
0,8/24/1921,"River Humber, England",[{u'geometry': {u'location_type': u'APPROXIMAT...,"(53.6339758, -0.2765141, United Kingdom)",53.634,-0.276514,United Kingdom
1,9/6/1921,"Paris, France",[{u'geometry': {u'location_type': u'APPROXIMAT...,"(48.856614, 2.3522219, France)",48.8566,2.35222,France
2,12/5/1921,"Near Murchinson River, Australia",[{u'geometry': {u'location_type': u'APPROXIMAT...,"(-25.8742799, 118.8406464, Australia)",-25.8743,118.841,Australia
3,1/14/1922,"Paris, France",[{u'geometry': {u'location_type': u'APPROXIMAT...,"(48.856614, 2.3522219, France)",48.8566,2.35222,France
4,2/21/1922,"Near Norfolk, Virginia",[{u'geometry': {u'location_type': u'APPROXIMAT...,"(36.8507689, -76.2858726, United States)",36.8508,-76.2859,United States
